In [27]:
# !nvidia-smi
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # this MUST come before any tf call.

## Seed for reproductibility

In [29]:
import numpy as np
import random
import tensorflow as tf

seed = 1337
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [30]:
import difflib
import os
import shutil
import SimpleITK as sitk
import scipy.spatial
from glob import glob
from typing import  List
from pprint import pprint  
import json
from skimage.transform import resize
import matplotlib.pyplot as plt
import os
import time
import warnings
import scipy
from datetime import datetime
import SimpleITK as sitk
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Cropping2D, ZeroPadding2D, BatchNormalization, Activation
from tensorflow.keras.activations import elu
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence, to_categorical
import tensorflow.keras as keras
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, confusion_matrix, multilabel_confusion_matrix
from tensorflow.keras.models import load_model
from helpers import *

In [31]:
base_path = '/your/root/path'
datasets_path = f'{base_path}/datasets/paper lesions extended'
models_path = f'{base_path}/modelos/paper lesions extended'
mni_T1_path = TEMPLATE_PATH = f'{base_path}/datasets/templates/mni_icbm152_t1_tal_nlin_sym_09a.nii'
clf_dataset = f'{datasets_path}/STANDARD_BY_CLASSIFICATION'
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [33]:
folders =  sorted(glob(f'{clf_dataset}/*/*'))
print(len(folders))
print(folders)

4
['/home/carlos.jimenez/ai-in-health/Stroke-Brain-Lesions/datasets/paper lesions extended/STANDARD_BY_CLASSIFICATION/test/Lacunar', '/home/carlos.jimenez/ai-in-health/Stroke-Brain-Lesions/datasets/paper lesions extended/STANDARD_BY_CLASSIFICATION/test/MCA', '/home/carlos.jimenez/ai-in-health/Stroke-Brain-Lesions/datasets/paper lesions extended/STANDARD_BY_CLASSIFICATION/train/Lacunar', '/home/carlos.jimenez/ai-in-health/Stroke-Brain-Lesions/datasets/paper lesions extended/STANDARD_BY_CLASSIFICATION/train/MCA']


## DataGenerator

In [34]:
NUM_EPOCHS = 100
IMG_SIZE = 128 # 224
BATCH_SIZE = 1024 # 64
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
LR = 1e-04
WD = 4e-03
TRAIN_PATH = f'{clf_dataset}/train'
TEST_PATH = f'{clf_dataset}/test'
class_mode = 'categorical'
loss = f'{class_mode}_crossentropy'
dense_layers = 3

# optimizer = tf.keras.optimizers.Adam(
#     learning_rate=tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=LR, first_decay_steps=300), 
#     decay=WD
# )
optimizer = tf.keras.optimizers.Adam(learning_rate=LR, decay=WD)

In [35]:
def scheduler(epoch, lr):
    """
    Learning rate scheduler function.

    Args:
        epoch (int): The current epoch number.
        lr (float): The current learning rate.

    Returns:
        float: The updated learning rate.

    """
    if epoch % 10 == 0:
        return lr * tf.math.exp(-0.1)
    return lr

In [36]:
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [37]:
def create_datagenerators(dataset, preprocessing_function, val_split=0.2, image_size=IMG_SIZE, batch_size=BATCH_SIZE):
    """
    Creates data generators for training and validation.

    Args:
        dataset (str): The path to the dataset directory.
        preprocessing_function (function): The preprocessing function to apply to the images.
        val_split (float): The fraction of data to use for validation.
        image_size (int): The target image size for resizing.
        batch_size (int): The batch size for the data generators.

    Returns:
        tuple: A tuple containing the training and validation data generators.

    """
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#         rescale=1./255,
        validation_split=val_split,
#         fill_mode="constant",
        preprocessing_function=preprocessing_function
    )

    # Create a generator for our training data
    train_generator = datagen.flow_from_directory(
        dataset,
        class_mode=class_mode,
        shuffle=True,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        seed=seed,
        subset="training"
    )

    # Create a generator for our validation data
    validation_generator = datagen.flow_from_directory(
        dataset,
        class_mode=class_mode,
        shuffle=True,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        seed=seed,
        subset="validation"
    )
  
    return train_generator, validation_generator

## Building a model

### VGG19

In [38]:
# Create a function which builds a Keras model
def create_model_vgg19(input_shape=INPUT_SHAPE):
    """
    Create a Keras model based on VGG19 architecture.

    Args:
        input_shape (tuple): The input shape of the model. Default is INPUT_SHAPE.

    Returns:
        tf.keras.Model: The created Keras model.

    """
    print("Building model...")
    # Setup the model layers
    base_model_VGG19 = tf.keras.applications.VGG19(include_top=False, 
                                                 weights='imagenet', 
                                                 input_shape=INPUT_SHAPE,
                                                 classifier_activation='relu')

    #  base vgg19 layers = 22
    base_model_VGG19.trainable = False    # IMPORTANT - Freeze the convolutional base 

    model = tf.keras.Sequential([
       base_model_VGG19,
       tf.keras.layers.Flatten()
    ])
    
    for i in range(dense_layers):
        model.add(tf.keras.layers.Dense(256, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.6))
    
    if class_mode == 'categorical':
        model.add(tf.keras.layers.Dense(2, activation='softmax'))
    elif class_mode == 'binary':
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(
      loss=loss,
      optimizer=optimizer,
      metrics=["accuracy"]
    )

    return model

### InceptionV3

In [39]:
def create_model_inceptionv3(input_shape=INPUT_SHAPE):
    """
    Create a Keras model based on InceptionV3 architecture.

    Args:
        input_shape (tuple): The input shape of the model. Default is INPUT_SHAPE.

    Returns:
        tf.keras.Model: The created Keras model.

    """
    print("Building model...")
    # Setup the model layers
    base_model_InceptionV3 = tf.keras.applications.InceptionV3(include_top=False, 
                                                 weights='imagenet', 
                                                 input_shape=INPUT_SHAPE,
                                                 classifier_activation='relu')
    
    #  base inceptionv3 layers = 311
    base_model_InceptionV3.trainable = False # IMPORTANT - Freeze the convolutional base

    for layer in  base_model_InceptionV3.layers[133:]:
#         print(layer.name)
        layer.trainable=True


    model = tf.keras.Sequential([
       base_model_InceptionV3,
       tf.keras.layers.Flatten()
    ])
    
    for i in range(dense_layers):
        model.add(tf.keras.layers.Dense(256, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.6))
    
    if class_mode == 'categorical':
        model.add(tf.keras.layers.Dense(2, activation='softmax'))
    elif class_mode == 'binary':
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(
      loss=loss,
      optimizer=optimizer,
      metrics=["accuracy"]
    )

    return model

### ResNet50V2

In [40]:
def create_model_resnet50v2(input_shape=INPUT_SHAPE):
    """
    Create a Keras model based on ResNet50V2 architecture.

    Args:
        input_shape (tuple): The input shape of the model. Default is INPUT_SHAPE.

    Returns:
        tf.keras.Model: The created Keras model.

    """
    print("Building model...")

    # Setup the model layers
    base_model_ResNet50V2 = tf.keras.applications.ResNet50V2(include_top=False, 
                                                 weights='imagenet', 
                                                 input_shape=INPUT_SHAPE,
                                                 classifier_activation='relu')
    #  base resnet50v2 layers = 190
    base_model_ResNet50V2.trainable = False # IMPORTANT - Freeze the convolutional base

    for layer in  base_model_ResNet50V2.layers[120:]:
#         print(layer.name)
        layer.trainable=True
#         print(layer.name)

    model = tf.keras.Sequential([
       base_model_ResNet50V2,
       tf.keras.layers.Flatten()
    ])
    
    for i in range(dense_layers):
        model.add(tf.keras.layers.Dense(256, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.6))
    
    if class_mode == 'categorical':
        model.add(tf.keras.layers.Dense(2, activation='softmax'))
    elif class_mode == 'binary':
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(
      loss=loss,
      optimizer=optimizer,
      metrics=["accuracy"]
    )

    return model

### MobileNetV2

In [41]:
def create_model_mobilenetv2(input_shape=INPUT_SHAPE):
    """
    Create a Keras model based on MobileNetV2 architecture.

    Args:
        input_shape (tuple): The input shape of the model. Default is INPUT_SHAPE.

    Returns:
        tf.keras.Model: The created Keras model.

    """
    print("Building model...")
    # Setup the model layers
    base_model_MobileNetV2 = tf.keras.applications.MobileNetV2(include_top=False, 
                                                 weights='imagenet', 
                                                 input_shape=INPUT_SHAPE,
                                                 classifier_activation='relu')
    #  base mobilenetv2 layers = 154
    base_model_MobileNetV2.trainable = False # IMPORTANT - Freeze the convolutional base
    
    for layer in base_model_MobileNetV2.layers[90:]:
#         print(layer.name)
        layer.trainable=True

    model = tf.keras.Sequential([
                               base_model_MobileNetV2,
                               tf.keras.layers.Flatten(),
    ])
    
    for i in range(dense_layers):
        model.add(tf.keras.layers.Dense(256, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.6))

    if class_mode == 'categorical':
        model.add(tf.keras.layers.Dense(2, activation='softmax'))
    elif class_mode == 'binary':
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(
      loss=loss,
      optimizer=optimizer,
      metrics=["accuracy"]
    )

    return model

## Create Models

In [42]:
with strategy.scope():
    model_vgg19 = create_model_vgg19()
# model_vgg19.summary()

Building model...


In [43]:
with strategy.scope():
    model_inceptionv3 = create_model_inceptionv3()
# model_inceptionv3.summary()

Building model...


In [44]:
with strategy.scope():
    model_resnet50v2 = create_model_resnet50v2()
# model_resnet50v2.summary()

Building model...


In [45]:
with strategy.scope():
    model_mobilenetv2 = create_model_mobilenetv2()
# model_mobilenetv2.summary()

Building model...


## Validation Model

### Callbacks


In [46]:
checkpoint_filepath = f"{models_path}/classification/inceptionV3_{NUM_EPOCHS}epocas_{len(model_inceptionv3.layers)}capas_lr_{str(LR)}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

In [47]:
def log_metrics(model, metrics):
    """
    Log and save the evaluation metrics of a model to a JSON file.

    Args:
        model (str): The name or identifier of the model.
        metrics (dict): A dictionary containing the evaluation metrics to be logged.

    Returns:
        None

    """
    save_path = f'{datasets_path}/results/{model}/metrics_{datetime.now().strftime("%Y%m%d%H%M-%S")}.json'
    with open(save_path, 'w') as fp:
        json.dump(str(metrics), fp)
        
    print(f'json saved to {save_path}')

# Training Models

In [48]:
train_generator, val_generator = create_datagenerators(
    TRAIN_PATH, 
    preprocessing_function=keras.applications.inception_v3.preprocess_input
)

training_steps = train_generator.n//train_generator.batch_size
validation_steps = val_generator.n//val_generator.batch_size

Found 6618 images belonging to 2 classes.
Found 1653 images belonging to 2 classes.


# InceptionV3
The network is trained using the InceptionV3 model.

In [55]:
history_inception = model_inceptionv3.fit(
    train_generator,
    epochs=NUM_EPOCHS,
    validation_data=val_generator,
    steps_per_epoch=training_steps,
    validation_steps=validation_steps,
    callbacks=[model_checkpoint_callback, lr_scheduler],
    # callbacks=[tensorboard],
    #class_weight=class_weight,
    shuffle=True
)

Epoch 1/100
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
6/6 [==============================] - 57s 2s/step - loss: 0.9655 - accuracy: 0.5195 - val_loss: 0.6643 - val_accuracy: 0.6064 - lr: 9.0484e-05
Epoch 2/100
6/6 [==============================] - 5s 952ms/step - loss: 0.7986 - accuracy: 0.5299 - val_loss: 0.6665 - val_accuracy: 0.6172 - lr: 9.0484e-05
Epoch 3/100
6/6 [==============================] - 5s 938ms/step - loss: 0.7343 - accuracy: 0.5458 - val_loss: 0.6588 - val_accuracy: 0.6748 - lr: 9.0484e-05
Epoch 4/100
6/6 [==============================] - 5s 939ms/step - loss: 0.7077 - accuracy: 0.5506 - val_loss: 0.6422 - val_accuracy: 0.7002 - lr: 9.0484e-05
Epoch 5/100
6/6 [==============================] - 5s 934ms/step - loss: 0.6743 - accuracy: 0.5788 - val_loss: 0.6314 - val_accuracy: 0.7158 - lr: 9.0484e-05
Epoch 6/100
6/6 [=========================

In [57]:
test_generator = create_testgenerator(
    TEST_PATH,
    class_mode=class_mode,
    preprocessing_function=keras.applications.inception_v3.preprocess_input,
    image_size=IMG_SIZE
)
y_true = test_generator.classes

Found 2270 images belonging to 2 classes.


In [58]:
# model_inceptionv3 = load_model(
#     f"{models_path}/classification/inceptionV3_{NUM_EPOCHS}epocas_{len(model_inceptionv3.layers)}capas_lr_{str(LR)}.h5",
#     compile=False
# )
model_name = 'inceptionV3'

In [59]:
y_pred = make_prediction(model_inceptionv3, test_generator, class_mode=class_mode, verbose=False)

2270/2270 [==============================] - 34s 12ms/step


In [60]:
set(y_pred)

{0, 1}

In [61]:
y_true

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

# Vgg19
The network is trained using the VGG19 model.

In [64]:
train_generator, val_generator = create_datagenerators(TRAIN_PATH, preprocessing_function=keras.applications.vgg19.preprocess_input)

training_steps = train_generator.n//train_generator.batch_size
validation_steps = val_generator.n//val_generator.batch_size

Found 6618 images belonging to 2 classes.
Found 1653 images belonging to 2 classes.


In [65]:
checkpoint_filepath = f"{models_path}/classification/vgg19_{NUM_EPOCHS}epocas_{len(model_vgg19.layers)}capas_lr_{str(LR)}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

In [66]:
# model_vgg19.load_weights("/content/drive/MyDrive/integradora_fiec/modelo_clasificacion/vgg19_40_epocas_15capas_lr_{str(LR)}.h5")
history_vgg19 = model_vgg19.fit(
    train_generator,
    epochs=NUM_EPOCHS,
    validation_data=val_generator,
    steps_per_epoch=training_steps,
    validation_steps=validation_steps,
    callbacks=[model_checkpoint_callback, lr_scheduler],
    #class_weight=class_weight,
     shuffle=True
)

Epoch 1/100
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
6/6 [==============================] - 15s 1s/step - loss: 3.5686 - accuracy: 0.5187 - val_loss: 0.6699 - val_accuracy: 0.6240 - lr: 3.3287e-05
Epoch 2/100
6/6 [==============================] - 6s 1s/step - loss: 3.0256 - accuracy: 0.5156 - val_loss: 0.6836 - val_accuracy: 0.5986 - lr: 3.3287e-05
Epoch 3/100
6/6 [==============================] - 6s 992ms/step - loss: 2.5896 - accuracy: 0.5356 - val_loss: 0.6764 - val_accuracy: 0.6201 - lr: 3.3287e-05
Epoch 4/100
6/6 [==============================] - 6s 1s/step - loss: 2.3002 - accuracy: 0.5374 - val_loss: 0.6376 - val_accuracy: 0.6621 - lr: 3.3287e-05
Epoch 5/100
6/6 [==============================] - 6s 1s/step - loss: 2.1370 - accuracy: 0.5279 - val_loss: 0.5906 - val_accuracy: 0.7012 - lr: 3.3287e-05
Epoch 6/100
6/6 [==============================] - 

In [68]:
test_generator = create_testgenerator(
    TEST_PATH,
    class_mode=class_mode,
    preprocessing_function=keras.applications.vgg19.preprocess_input,
    image_size=IMG_SIZE
)
y_true = test_generator.classes

Found 2291 images belonging to 2 classes.


In [69]:
# model_vgg19 = load_model(
#     f'{models_path}/classification/vgg19_{NUM_EPOCHS}epocas_{len(model_vgg19.layers)}capas_lr_{str(LR)}.h5',
#     compile=False
# )
model_name = 'vgg19'

In [70]:
y_pred = make_prediction(model_vgg19, test_generator, class_mode=class_mode)

2291/2291 [==============================] - 12s 4ms/step


# ResNet50v2
The network is trained using the ResNet50v2 model.


In [47]:
train_generator, val_generator = create_datagenerators(TRAIN_PATH, preprocessing_function=keras.applications.resnet_v2.preprocess_input)

training_steps = train_generator.n//train_generator.batch_size
validation_steps = val_generator.n//val_generator.batch_size

Found 15029 images belonging to 2 classes.
Found 3756 images belonging to 2 classes.


In [48]:
checkpoint_filepath = f"{models_path}/classification/resnet50v2_{NUM_EPOCHS}epocas_{len(model_resnet50v2.layers)}capas_lr_{str(LR)}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

In [ ]:
history_resnet = model_resnet50v2.fit(train_generator,
    epochs=NUM_EPOCHS,
    validation_data=val_generator,
    steps_per_epoch=training_steps,
    validation_steps=validation_steps,
    callbacks=[model_checkpoint_callback, lr_scheduler],
    #class_weight=class_weight,
    shuffle=True
                                   
)

Epoch 1/100
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
14/14 [==============================] - ETA: 0s - loss: 2.1544 - accuracy: 0.5069

In [ ]:
test_generator = create_testgenerator(
    TEST_PATH,
    class_mode=class_mode,
    preprocessing_function=keras.applications.resnet_v2.preprocess_input,
    image_size=IMG_SIZE
)
y_true = test_generator.classes

In [ ]:
# model_resnet50v2 = load_model(
#     f"{models_path}/classification/resnet50v2_{NUM_EPOCHS}epocas_{len(model_resnet50v2.layers)}capas_lr_{str(LR)}.h5",
#     compile=False
# )
model_name = 'resnet50v2'

In [ ]:
y_pred = make_prediction(model_resnet50v2, test_generator, class_mode=class_mode)

# MobileNetv2
The network is trained using the MobileNetv2 model.


In [ ]:
train_generator, val_generator = create_datagenerators(TRAIN_PATH, preprocessing_function=keras.applications.mobilenet_v2.preprocess_input)

training_steps = train_generator.n//train_generator.batch_size
validation_steps = val_generator.n//val_generator.batch_size

In [ ]:
checkpoint_filepath = f"{models_path}/classification/mobilenetv2_{NUM_EPOCHS}epocas_{len(model_mobilenetv2.layers)}capas_lr_{str(LR)}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

In [ ]:
history_mobile = model_mobilenetv2.fit(
    train_generator,
    epochs=NUM_EPOCHS,
    validation_data=val_generator,
    steps_per_epoch=training_steps,
    validation_steps=validation_steps,
    callbacks=[model_checkpoint_callback, lr_scheduler],
    #class_weight=class_weight,
     shuffle=True
)

In [ ]:
test_generator = create_testgenerator(
    TEST_PATH,
    class_mode=class_mode,
    preprocessing_function=keras.applications.mobilenet_v2.preprocess_input,
    image_size=IMG_SIZE
)
y_true = test_generator.classes

In [ ]:
# model_mobilenetv2 = load_model(
#     f"{models_path}/classification/mobilenetv2_{NUM_EPOCHS}epocas_{len(model_mobilenetv2.layers)}capas_lr_{str(LR)}.h5",
#     compile=False
# )
model_name = 'mobilenetV2'

In [ ]:
y_pred = make_prediction(model_mobilenetv2, test_generator, class_mode=class_mode)